In [1]:
import os
import pg8000          #pg8000 access SQL databases
import pandas as pd    #pandas will be needed to work in a dataframe

In [2]:
#code from Agata
#these are nice functions to open LIMS, make a query and then close LIMS after

def _connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    conn = pg8000.connect(user=user, host=host, database=database, password=password, port=port)
    return conn, conn.cursor()

def _select(cursor, query):
    cursor.execute(query)
    columns = [ d[0] for d in cursor.description ]
    return [ dict(zip(columns, c)) for c in cursor.fetchall() ]

def limsquery(query, user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    """A function that takes a string containing a SQL query, connects to the LIMS database and outputs the result."""
    conn, cursor = _connect(user, host, database, password, port)
    try:
        results = _select(cursor, query)
    finally:
        
        #THESE ARE IMPORTANT!!!!!!
        #Every query needs to be closed when done
        cursor.close()             
        conn.close()
    return results


#this last function will take our query results and put them in a dataframe so that they are easy to work with
def get_lims_dataframe(query):
    '''Return a dataframe with lims query'''
    result = limsquery(query)
    try:
        data_df = pd.DataFrame(data=result, columns=result[0].keys())
    except IndexError:
        print "Could not find results for your query."
        data_df = pd.DataFrame()
    return data_df

# Make user report

In [3]:
def user_df(P_number):
    """Return a pandas dataframe with all patched cells for a specified user.
     
    Parameters
    ----------
    P_number : A string. Corresponds to the user's P number in the form of P1, P2, P3, etc.
       
    Returns
    -------
    A pandas dataframe
    """
    user = master_df[master_df["patched_cell_container"].str.contains(P_number)]
    return user


In [4]:
def cell_count(P_number, df):
    """Return the number of cells in a specified dataframe for a specified user.
     
    Parameters
    ----------
    P_number : A string. Corresponds to the user's P number in the form of P1, P2, P3, etc.
    df: a pandas dataframe
    
    Returns
    -------
    An integer cell number
    """
    user = df[df["patched_cell_container"].str.contains(P_number)]
    return user["patched_cell_container"].count()
    

In [5]:
def gen_filter(df, col, cond):
    filtered = df[df[col] == cond]
    return filtered

In [6]:
def per_user(P_number):
    """Returns a summary of patcher metrics for a specified user.
     
    Parameters
    ----------
    P_number : A string. Corresponds to the user's P number in the form of P1, P2, P3, etc.
        
    Returns
    -------
    Returns None
    Prints a summary of patcher metrics, including total cells patched, cells that passed RNA seq QC, and QC pass percentage
    """
    print "Total cells patched:", cell_count(P_number,master_df)
    print "Cells that passed RNA seq QC:", cell_count(P_number,qcpass)
    print "QC pass percentage:", float((cell_count(P_number,qcpass)))/float((cell_count(P_number,master_df)))
    print "Cells imaged at 63x:", cell_count(P_number,imagego)
    
    

In [7]:
query = "WITH do_63x(cell_id, go) AS (SELECT DISTINCT cell.id, array_to_string(array_agg(DISTINCT tag.name), ' _AND_ ') \
FROM specimens cell JOIN ephys_roi_results err ON err.id = cell.ephys_roi_result_id \
JOIN specimen_tags_specimens sptagsp ON sptagsp.specimen_id = cell.id \
JOIN specimen_tags tag ON tag.id = sptagsp.specimen_tag_id AND tag.id in (602120185,602122082) \
GROUP BY cell.id ORDER BY 1) \
SELECT s.patched_cell_container, err.recording_date, do_63x.go AS go_no_go_63x, ra.failed AS RNA_seq_QC_Failed, s.name \
FROM specimens s \
LEFT JOIN projects proj ON s.project_id = proj.id \
LEFT JOIN do_63x ON do_63x.cell_id = s.id \
LEFT JOIN ephys_roi_results err ON s.ephys_roi_result_id = err.id \
LEFT JOIN rna_amplification_inputs rai on rai.sample_id = s.id \
LEFT JOIN rna_amplifications ra on ra.id = rai.rna_amplification_id \
WHERE proj.code <> 'mMPATCH' AND \
s.patched_cell_container IS NOT NULL \
AND err.recording_date > '2017-10-01'"

#AND (err.recording_date > '2017-10-01' OR err.recording_date IS NULL)
#AND err.recording_date > '2017-10-01'

master_df = get_lims_dataframe(query)
#master_df.sort_values("patched_cell_container", axis = 0, inplace = True)
master_df.head()

,rna_seq_qc_failed,patched_cell_container,go_no_go_63x,name,recording_date
0,False,P8S4_171002_351_A01,63x go,Oxtr-T2A-Cre;Ai14-351467.05.02.01,2017-10-02 17:55:51
1,False,P9S4_171002_401_A01,63x go,Oxtr-T2A-Cre;Ai14-351467.03.01.01,2017-10-02 18:00:47
2,True,P8S4_171002_352_A01,63x no go,Oxtr-T2A-Cre;Ai14-351467.05.02.02,2017-10-02 18:17:44
3,False,P2S4_171002_051_A01,63x go,Nos1-CreERT2;Sst-IRES-FlpO;Ai65-350934.04.01.01,2017-10-02 18:29:21
4,False,P8S4_171002_353_A01,63x no go,Oxtr-T2A-Cre;Ai14-351467.05.02.03,2017-10-02 18:47:06


In [8]:
qcpass = gen_filter(master_df,'rna_seq_qc_failed', False)

In [9]:
imagego = gen_filter(master_df,'go_no_go_63x', '63x go')

In [23]:
per_user('P8')

Total cells patched: 718
Cells that passed RNA seq QC: 511
QC pass percentage: 0.711699164345
Cells imaged at 63x: 169


In [11]:
user_df('P8')

,rna_seq_qc_failed,patched_cell_container,go_no_go_63x,name,recording_date
0,False,P8S4_171002_351_A01,63x go,Oxtr-T2A-Cre;Ai14-351467.05.02.01,2017-10-02 17:55:51
2,True,P8S4_171002_352_A01,63x no go,Oxtr-T2A-Cre;Ai14-351467.05.02.02,2017-10-02 18:17:44
4,False,P8S4_171002_353_A01,63x no go,Oxtr-T2A-Cre;Ai14-351467.05.02.03,2017-10-02 18:47:06
7,False,P8S4_171002_354_A01,63x go,Chrna2-Cre_OE25;Ai14-351067.04.02.01,2017-10-02 20:39:22
8,False,P8S4_171002_355_A01,63x go,Chrna2-Cre_OE25;Ai14-351067.04.02.02,2017-10-02 21:12:16
13,False,P8S4_171002_356_A01,63x go,Chrna2-Cre_OE25;Ai14-351067.03.02.01,2017-10-02 22:10:03
18,True,P8S4_171002_357_A01,63x no go,Chrna2-Cre_OE25;Ai14-351067.03.02.02,2017-10-02 22:35:02
22,False,P8S4_171003_351_A01,63x go,Gad2-IRES-Cre;Ai14-350674.04.02.01,2017-10-03 16:37:01
24,True,P8S4_171003_352_A01,63x no go,Gad2-IRES-Cre;Ai14-350674.04.02.02,2017-10-03 16:59:16
31,False,P8S4_171003_353_A01,63x go,Chrna2-Cre_OE25;Ai14-351070.04.02.01,2017-10-03 18:07:38


lims 20x image link